In [1]:
from owlready2 import Thing, get_ontology, AllDisjoint, sync_reasoner_pellet, Imp, FunctionalProperty
from owlready2 import *

# Create a new ontology
onto = get_ontology("http://example.org/onto#")

# Define classes for the ontology based on the CSV file contents
with onto:
    class TVShow(Thing): pass
    class Episode(Thing): pass
    class User(Thing): pass
    class Comment(Thing): pass
    class Rating(Thing): pass
    class Emotion(Thing): pass

    # Define relationships
    class posts_by(Comment >> User): pass
    class posts_comment(User >> Comment):
        inverse_property = posts_by
    class comments_on(Comment >> Episode): pass

    class follows(User >> TVShow): pass
    class has_started(User >> TVShow): pass
    class has_stopped(User >> TVShow): pass
    class is_stopped_by(TVShow >> User): 
        domain = [TVShow]
        range = [User]
        inverse_property = has_stopped
    class has_watched(User >> Episode): pass

    class rated_by(Rating >> User): pass
    class rates(Rating >> Episode): pass

    class expressed_by(Emotion >> User): pass
    class evokes(Emotion >> Episode):  pass

    # Define properties
    class has_username(User >> str, FunctionalProperty): pass
    class has_follow_date(User >> str, FunctionalProperty): pass
    class has_follow_status(User >> str, FunctionalProperty): pass

    class has_name(TVShow >> str, FunctionalProperty): pass

    class has_season_number(Episode >> int, FunctionalProperty): pass
    class has_rewatched_count(Episode >> int, FunctionalProperty): pass
    class has_number(Episode >> int, FunctionalProperty): pass
    class has_date_of_seen(Episode >> str, FunctionalProperty): pass

    class has_note(Rating >> int, FunctionalProperty): pass
    class has_rating_date(Rating >> str, FunctionalProperty): pass

    class has_text(Comment >> str, FunctionalProperty): pass
    class has_comment_date(Comment >> str, FunctionalProperty): pass
    class has_last_update(Comment >> str, FunctionalProperty): pass
    class has_number_of_likes(Comment >> int, FunctionalProperty): pass

    class has_emotion(Emotion >> int, FunctionalProperty): pass
    class has_emotion_date(Emotion >> str, FunctionalProperty): pass

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


In [2]:
# Reasoning
sync_reasoner_pellet([onto], infer_property_values = True, infer_data_property_values = True, debug=3)

* Owlready2 * Running Pellet...
    java -Xmx2000M -cp C:\Users\romai\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\owlready2\pellet\antlr-3.2.jar;C:\Users\romai\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\owlready2\pellet\antlr-runtime-3.2.jar;C:\Users\romai\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\owlready2\pellet\aterm-java-1.6.jar;C:\Users\romai\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\owlready2\pellet\commons-codec-1.6.jar;C:\Users\romai\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\owlready2\pellet\httpclient-4.2.3.jar;C:\Users\romai\AppData\Local\Packages\PythonSoftwareFoundation.Pyth

In [3]:
import csv

def load_data(filename, process_row):
    with open(filename, 'rt', encoding='utf-8') as file:
        reader = csv.DictReader(file, delimiter=',')
        count = 0
        for row in reader:
            if count > 0 and count%100000 == 0:
                print(f'Processed {count} rows')
            process_row(row)
            count += 1

In [5]:
import datetime

def process_tv_shows_episodes(row):
    tv_show = onto.TVShow(name=row['tv_show_name'])
    tv_show.has_name = row['tv_show_name']

    episode = onto.Episode(name=row['episode_id'])
    episode.has_season_number = int(row['episode_season_number'])
    episode.has_number = int(row['episode_number'])
    episode.has_rewatched_count = int(row['rewatched_count'])
    episode.has_date_of_seen = datetime.datetime.strptime(row['created_at'], '%Y-%m-%d %H:%M:%S').date()

    user = onto.User(name=row['user_id'])
    user.has_username = row['user_id']
    user.has_started.append(tv_show) # user has started watching the tv show because they have seen at least one episode
    user.has_watched.append(episode) # user has watched the episode

path = './data/seen_episode_modified.csv'
load_data(path, process_tv_shows_episodes)
print(f'Loaded {len(onto.TVShow.instances())} TV shows and {len(onto.Episode.instances())} episodes seen and {len(onto.User.instances())} users')

Loaded 106 TV shows and 5227 episodes seen and 1 users


In [6]:
# Reasoning
sync_reasoner_pellet([onto], infer_property_values = True, infer_data_property_values = True, debug=3)

* Owlready2 * Running Pellet...
    java -Xmx2000M -cp C:\Users\romai\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\owlready2\pellet\antlr-3.2.jar;C:\Users\romai\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\owlready2\pellet\antlr-runtime-3.2.jar;C:\Users\romai\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\owlready2\pellet\aterm-java-1.6.jar;C:\Users\romai\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\owlready2\pellet\commons-codec-1.6.jar;C:\Users\romai\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\owlready2\pellet\httpclient-4.2.3.jar;C:\Users\romai\AppData\Local\Packages\PythonSoftwareFoundation.Pyth

OwlReadyInconsistentOntologyError: Java error message is: ERROR: Ontology is inconsistent, run "pellet explain" to get the reason

This is the output of `pellet explain`: 
 
Problem parsing file:/C:/Users/romai/AppData/Local/Temp/tmp29l90btr
Use -v for detail.


In [5]:
def process_episodes_ratings(row):
    episode = onto.Episode(name=row['episode_id'])
    user = onto.User(name=row['user_id'])
    rating = onto.Rating(name=row['vote_key'])
    rating.has_note = int(row['note'])
    rating.rated_by.append(user)
    rating.rates.append(episode)
    rating.has_rating_date = datetime.datetime.strptime(row['created_at'], '%Y-%m-%d %H:%M:%S').date()

path = './data/ratings_episode_votes_modified.csv'
load_data(path, process_episodes_ratings)
print(f'Loaded {len(onto.Rating.instances())} ratings')

Loaded 339 ratings


In [6]:
def process_tv_shows_followings(row):
    tv_show = onto.TVShow(name=row['tv_show_name'])

    user = onto.User(name=row['user_id'])
    user.follows.append(tv_show)
    if row['active'] == '0' or row['archived'] == '1': # user has stopped following the tv show
        user.has_stopped.append(tv_show)
        tv_show.is_stopped_by.append(user)

path = './data/followed_tv_show_modified.csv'
load_data(path, process_tv_shows_followings)
print(f'Loaded {len(onto.User.instances())} users and {len(onto.TVShow.instances())} TV shows')

Loaded 1 users and 119 TV shows


In [7]:
def process_episodes_comments(row):
    episode = onto.Episode(name=row['episode_id'])
    user = onto.User(name=row['user_id'])
    comment = onto.Comment(name=row['comment_id'])
    comment.has_text = row['comment']
    comment.posts_by.append(user)
    comment.comments_on.append(episode)
    comment.has_number_of_likes = int(row['nb_likes'])
    comment.has_last_update = datetime.datetime.strptime(row['updated_at'], '%Y-%m-%d %H:%M:%S').date()
    comment.has_comment_date = datetime.datetime.strptime(row['created_at'], '%Y-%m-%d %H:%M:%S').date()

path = './data/episode_comment_modified.csv'
load_data(path, process_episodes_comments)
print(f'Loaded {len(onto.Comment.instances())} comments')

Loaded 35 comments


In [8]:
def process_episodes_emotions(row):
    episode = onto.Episode(name=row['episode_id'])
    user = onto.User(name=row['user_id'])
    emotion = onto.Emotion(name=row['emotion_vote_id'])
    emotion.has_emotion = row['emotion_id']
    emotion.expressed_by.append(user)
    emotion.evokes.append(episode)
    emotion.has_emotion_date = datetime.datetime.strptime(row['created_at'], '%Y-%m-%d %H:%M:%S').date()

path = './data/emotions_episode_votes_modified.csv'
load_data(path, process_episodes_emotions)
print(f'Loaded {len(onto.Emotion.instances())} emotions')

Loaded 341 emotions


In [9]:
# Reasoning
sync_reasoner_pellet([onto], infer_property_values = True, infer_data_property_values = True, debug=3)

* Owlready2 * Running Pellet...
    java -Xmx2000M -cp C:\Users\romai\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\owlready2\pellet\antlr-3.2.jar;C:\Users\romai\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\owlready2\pellet\antlr-runtime-3.2.jar;C:\Users\romai\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\owlready2\pellet\aterm-java-1.6.jar;C:\Users\romai\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\owlready2\pellet\commons-codec-1.6.jar;C:\Users\romai\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\owlready2\pellet\httpclient-4.2.3.jar;C:\Users\romai\AppData\Local\Packages\PythonSoftwareFoundation.Pyth

OwlReadyInconsistentOntologyError: Java error message is: ERROR: Ontology is inconsistent, run "pellet explain" to get the reason

This is the output of `pellet explain`: 
 
Problem parsing file:/C:/Users/romai/AppData/Local/Temp/tmplocfbbga
Use -v for detail.


In [9]:
# Save the ontology
onto.save(file = "tvtime_ontology_bug.owl", format = "rdfxml")

# Création de classes plus complexes

In [10]:
with onto:
    class User(Thing):
        equivalent_to = [User & (has_started.some(TVShow) & has_watched.some(Episode) & follows.some(TVShow) & has_stopped.some(TVShow))]

    class TVShow(Thing):
        equivalent_to = [TVShow & (follows.some(User) & has_started.some(User) & is_stopped_by.some(User))]

    class Episode(Thing):
        equivalent_to = [Episode & (has_watched.some(User) & rates.some(Rating) & comments_on.some(Comment) & evokes.some(Emotion))]

    class Comment(Thing):
        equivalent_to = [Comment & (posts_by.some(User) & comments_on.some(Episode))]

    class Rating(Thing):
        equivalent_to = [Rating & (rated_by.some(User) & rates.some(Episode))]

    class Emotion(Thing):
        equivalent_to = [Emotion & (expressed_by.some(User) & evokes.some(Episode))]

In [11]:
sync_reasoner_pellet([onto], infer_property_values = True, infer_data_property_values = True, debug=True)

* Owlready2 * Running Pellet...
    java -Xmx2000M -cp C:\Users\romai\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\owlready2\pellet\antlr-3.2.jar;C:\Users\romai\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\owlready2\pellet\antlr-runtime-3.2.jar;C:\Users\romai\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\owlready2\pellet\aterm-java-1.6.jar;C:\Users\romai\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\owlready2\pellet\commons-codec-1.6.jar;C:\Users\romai\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\owlready2\pellet\httpclient-4.2.3.jar;C:\Users\romai\AppData\Local\Packages\PythonSoftwareFoundation.Pyth

In [9]:
with onto:

    class TVShowEngageUser(Thing):
        # A tv show engages a user if the user has started watching the tv show, has watched at least one episode and has not stopped watching the tv show
        # equivalent_to = [TVShow & (has_started.some(User) & has_watched.some(Episode) & Not(has_stopped.some(User)))]
        rule1 = Imp()
        rule1.set_as_rule('''
            has_started(?user, ?tv_show),
            has_watched(?user, ?episode)
            -> TVShowEngageUser(?tv_show)''')

sync_reasoner_pellet([onto], infer_property_values = True, infer_data_property_values = True, debug=True)

print(f'Number of TV shows that engage a user: {len(onto.TVShowEngageUser.instances())} : ')
for tv_show in onto.TVShowEngageUser.instances():
    print(f'{tv_show.titre} engages a user')

ValueError: Cannot find entity 'TVShowEngageUser'!